<a href="https://colab.research.google.com/github/adenurchalisa/Automatic-Photo-Clustering-System-Optimization-HDBSCAN/blob/main/notebooks/2_Eksperimen_HDBSCAN_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cek GPU (untuk informasi saja)
print("🔍 GPU Info:")
!nvidia-smi --query-gpu=name,memory.total --format=csv

# Install hdbscan dengan optimasi
!pip install hdbscan -q

print("\n✅ Installation complete!")
print("💻 Using CPU multi-core mode (fast enough for our dataset)")

🔍 GPU Info:
name, memory.total [MiB]
Tesla T4, 15360 MiB

✅ Installation complete!
💻 Using CPU multi-core mode (fast enough for our dataset)


In [ ]:
import os
import time
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Clustering
import hdbscan

# Preprocessing & Evaluation
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Google Colab
from google.colab import drive

# Cek CPU cores
import multiprocessing
n_cores = multiprocessing.cpu_count()

print("✅ All libraries imported!")
print(f"💻 Available CPU cores: {n_cores}")
print(f"   HDBSCAN will use all {n_cores} cores for parallel processing")

✅ All libraries imported!
💻 Available CPU cores: 2
   HDBSCAN will use all 2 cores for parallel processing


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
EMBEDDINGS_PATH = "/content/drive/MyDrive/OTW S.KOM/Embeddings/embeddings_data.pkl"

OUTPUT_DIR = "/content/drive/MyDrive/OTW S.KOM/Eksperimen 2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

PARAM_GRID = {
    'min_cluster_size': [10, 15, 20, 25, 30, 40, 50, 75, 100],
    'min_samples': [1, 3, 5, 10, 15, 20, 25, None],
    'cluster_selection_method': ['eom', 'leaf']
}

total_combinations = (
    len(PARAM_GRID['min_cluster_size']) *
    len(PARAM_GRID['min_samples']) *
    len(PARAM_GRID['cluster_selection_method'])
)

print("✅ Configuration set!")
print(f"   - Embeddings path: {EMBEDDINGS_PATH}")
print(f"   - Output directory: {OUTPUT_DIR}")
print(f"\n📊 Parameter Grid:")
print(f"   - min_cluster_size: {PARAM_GRID['min_cluster_size']}")
print(f"   - min_samples: {PARAM_GRID['min_samples']}")
print(f"   - cluster_selection_method: {PARAM_GRID['cluster_selection_method']}")
print(f"   - Total combinations per embedding type: {total_combinations}")
print(f"   - Total experiments (original + normalized): {total_combinations * 2}")

✅ Configuration set!
   - Embeddings path: /content/drive/MyDrive/OTW S.KOM/Embeddings/embeddings_data.pkl
   - Output directory: /content/drive/MyDrive/OTW S.KOM/Eksperimen 2

📊 Parameter Grid:
   - min_cluster_size: [10, 15, 20, 25, 30, 40, 50, 75, 100]
   - min_samples: [1, 3, 5, 10, 15, 20, 25, None]
   - cluster_selection_method: ['eom', 'leaf']
   - Total combinations per embedding type: 144
   - Total experiments (original + normalized): 288


In [ ]:
def load_embeddings(file_path):
    """Load embeddings dari file pickle."""
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data['embeddings'], data['metadata'], data['stats']

# Load
print("🔄 Loading embeddings...")
embeddings, metadata, stats = load_embeddings(EMBEDDINGS_PATH)

print("✅ Loaded successfully!")
print(f"\n📊 Dataset:")
print(f"   - Total faces: {embeddings.shape[0]:,}")
print(f"   - Embedding dim: {embeddings.shape[1]}")
print(f"   - Original images: {stats['total_images']:,}")
print(f"   - Images with faces: {stats['images_with_faces']:,}")

🔄 Loading embeddings...
✅ Loaded successfully!

📊 Dataset:
   - Total faces: 12,715
   - Embedding dim: 512
   - Original images: 2,533
   - Images with faces: 2,365


In [ ]:
# ============================================================
# CELL 6: Core Functions
# ============================================================

def run_hdbscan(embeddings, min_cluster_size, min_samples,
                metric='euclidean', cluster_selection_method='eom'):
    """
    Run HDBSCAN clustering dengan CPU multi-core.
    """
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric=metric,
        cluster_selection_method=cluster_selection_method,
        core_dist_n_jobs=-1,  # Semua CPU cores
        algorithm='best'
    )

    labels = clusterer.fit_predict(embeddings)

    # Stats
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    noise_ratio = n_noise / len(labels) * 100

    # Metrics
    if n_clusters > 1:
        mask = labels != -1
        if mask.sum() > n_clusters:
            sil = silhouette_score(embeddings[mask], labels[mask])
            dbi = davies_bouldin_score(embeddings[mask], labels[mask])
        else:
            sil, dbi = -1, -1
    else:
        sil, dbi = -1, -1

    return {
        'labels': labels,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'noise_ratio': noise_ratio,
        'silhouette_score': sil,
        'davies_bouldin_index': dbi,
        'params': {
            'min_cluster_size': min_cluster_size,
            'min_samples': min_samples,
            'metric': metric,
            'cluster_selection_method': cluster_selection_method
        }
    }


def get_cluster_distribution(labels):
    """Get cluster size distribution."""
    unique, counts = np.unique(labels, return_counts=True)
    df = pd.DataFrame({'cluster_id': unique, 'count': counts})
    return df.sort_values('count', ascending=False).reset_index(drop=True)


print("✅ Functions defined!")

✅ Functions defined!


In [ ]:
# Normalize embeddings
embeddings_normalized = normalize(embeddings, norm='l2')

# Verify
orig_norms = np.linalg.norm(embeddings, axis=1)
norm_norms = np.linalg.norm(embeddings_normalized, axis=1)

print("📊 Embedding Comparison:")
print("=" * 50)
print(f"\n🔹 ORIGINAL:")
print(f"   L2 Norm range: [{orig_norms.min():.4f}, {orig_norms.max():.4f}]")
print(f"   Mean: {embeddings.mean():.6f}, Std: {embeddings.std():.6f}")

print(f"\n🔹 NORMALIZED (L2):")
print(f"   L2 Norm range: [{norm_norms.min():.4f}, {norm_norms.max():.4f}]")
print(f"   Mean: {embeddings_normalized.mean():.6f}, Std: {embeddings_normalized.std():.6f}")

print("\n✅ Preprocessing complete!")

📊 Embedding Comparison:

🔹 ORIGINAL:
   L2 Norm range: [6.2935, 30.1433]
   Mean: 0.003600, Std: 0.957149

🔹 NORMALIZED (L2):
   L2 Norm range: [1.0000, 1.0000]
   Mean: 0.000122, Std: 0.044194

✅ Preprocessing complete!


In [ ]:
# ============================================================
# CELL 8: Run Extended Grid Search
# ============================================================

def run_grid_search(embeddings, param_grid, embedding_type='original'):
    """Run grid search dengan progress tracking."""
    all_results = []

    total = (len(param_grid['min_cluster_size']) *
             len(param_grid['min_samples']) *
             len(param_grid['cluster_selection_method']))

    print(f"\n🔍 GRID SEARCH: {embedding_type.upper()}")
    print("=" * 70)

    start_time = time.time()
    idx = 0
    best_sil = -1
    best_result = None

    for mcs in param_grid['min_cluster_size']:
        for ms in param_grid['min_samples']:
            for csm in param_grid['cluster_selection_method']:
                idx += 1
                t0 = time.time()

                result = run_hdbscan(embeddings, mcs, ms, 'euclidean', csm)
                result['embedding_type'] = embedding_type
                all_results.append(result)

                # Track best
                if result['silhouette_score'] > best_sil:
                    best_sil = result['silhouette_score']
                    best_result = result

                # Progress
                t1 = time.time() - t0
                sil = result['silhouette_score']
                sil_str = f"{sil:.4f}" if sil != -1 else "N/A"
                star = "⭐" if sil > 0.35 else "  "

                print(f"{star}[{idx:3d}/{total}] mcs={mcs:3d}, ms={str(ms):4s}, "
                      f"csm={csm:4s} → clusters={result['n_clusters']:3d}, "
                      f"noise={result['noise_ratio']:5.1f}%, sil={sil_str} ({t1:.1f}s)")

    elapsed = time.time() - start_time
    print(f"\n✅ Completed in {elapsed:.1f}s ({elapsed/60:.1f} min)")
    print(f"🏆 Best Silhouette: {best_sil:.4f}")

    return all_results, best_result


# ============================================================
# RUN GRID SEARCH
# ============================================================
print("🚀 EXPERIMENT 1 EXTENDED: Euclidean Distance")
print("=" * 70)
print(f"   Dataset: {embeddings.shape[0]:,} faces, {embeddings.shape[1]} dims")
print("=" * 70)

# Run 1: Original
print("\n" + "📊 "*15)
print("📊 RUN 1: ORIGINAL EMBEDDINGS")
print("📊 "*15)
all_orig, best_orig = run_grid_search(embeddings, PARAM_GRID, 'original')

# Run 2: Normalized
print("\n" + "📊 "*15)
print("📊 RUN 2: NORMALIZED EMBEDDINGS")
print("📊 "*15)
all_norm, best_norm = run_grid_search(embeddings_normalized, PARAM_GRID, 'normalized')

print("\n✅ All Grid Searches Complete!")

🚀 EXPERIMENT 1 EXTENDED: Euclidean Distance
   Dataset: 12,715 faces, 512 dims

📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 
📊 RUN 1: ORIGINAL EMBEDDINGS
📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 

🔍 GRID SEARCH: ORIGINAL
  [  1/144] mcs= 10, ms=1   , csm=eom  → clusters=122, noise= 31.0%, sil=0.2259 (172.9s)


KeyboardInterrupt: 

In [ ]:
# ============================================================
# CELL 9: Compare & Analyze Results
# ============================================================

# Combine results
def to_dataframe(results):
    rows = []
    for r in results:
        rows.append({
            'embedding_type': r['embedding_type'],
            'min_cluster_size': r['params']['min_cluster_size'],
            'min_samples': r['params']['min_samples'],
            'cluster_selection_method': r['params']['cluster_selection_method'],
            'n_clusters': r['n_clusters'],
            'noise_ratio': r['noise_ratio'],
            'silhouette_score': r['silhouette_score'],
            'davies_bouldin_index': r['davies_bouldin_index']
        })
    return pd.DataFrame(rows)

df_all = pd.concat([to_dataframe(all_orig), to_dataframe(all_norm)], ignore_index=True)

# ============================================================
# COMPARISON
# ============================================================
print("\n" + "="*70)
print("📊 COMPARISON: ORIGINAL vs NORMALIZED")
print("="*70)

print(f"\n{'Metric':<25} {'Original':>15} {'Normalized':>15}")
print("-"*55)
print(f"{'Best Silhouette':<25} {best_orig['silhouette_score']:>15.4f} {best_norm['silhouette_score']:>15.4f}")
print(f"{'Best DBI':<25} {best_orig['davies_bouldin_index']:>15.4f} {best_norm['davies_bouldin_index']:>15.4f}")
print(f"{'N Clusters':<25} {best_orig['n_clusters']:>15} {best_norm['n_clusters']:>15}")
print(f"{'Noise Ratio':<25} {best_orig['noise_ratio']:>14.2f}% {best_norm['noise_ratio']:>14.2f}%")
print(f"{'min_cluster_size':<25} {best_orig['params']['min_cluster_size']:>15} {best_norm['params']['min_cluster_size']:>15}")
print(f"{'min_samples':<25} {str(best_orig['params']['min_samples']):>15} {str(best_norm['params']['min_samples']):>15}")
print(f"{'CSM':<25} {best_orig['params']['cluster_selection_method']:>15} {best_norm['params']['cluster_selection_method']:>15}")

# Determine winner
print("\n" + "="*70)
if best_norm['silhouette_score'] > best_orig['silhouette_score']:
    winner, winner_result, winner_emb = 'NORMALIZED', best_norm, embeddings_normalized
    diff = best_norm['silhouette_score'] - best_orig['silhouette_score']
    print(f"🏆 WINNER: NORMALIZED (+{diff:.4f} silhouette)")
else:
    winner, winner_result, winner_emb = 'ORIGINAL', best_orig, embeddings
    diff = best_orig['silhouette_score'] - best_norm['silhouette_score']
    print(f"🏆 WINNER: ORIGINAL (+{diff:.4f} silhouette)")
print("="*70)

# Top 10
print("\n📊 TOP 10 CONFIGURATIONS:")
df_valid = df_all[df_all['silhouette_score'] != -1]
print(df_valid.nlargest(10, 'silhouette_score').to_string(index=False))

In [ ]:
# ============================================================
# CELL 10: Visualization
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Extended Grid Search: Euclidean Distance', fontsize=14, fontweight='bold')

df_valid = df_all[df_all['silhouette_score'] != -1]

# Plot 1: Histogram
ax1 = axes[0, 0]
for et, c in [('original', 'steelblue'), ('normalized', 'coral')]:
    subset = df_valid[df_valid['embedding_type'] == et]
    ax1.hist(subset['silhouette_score'], bins=20, alpha=0.6, label=et.title(), color=c)
ax1.axvline(best_orig['silhouette_score'], color='steelblue', ls='--', lw=2)
ax1.axvline(best_norm['silhouette_score'], color='coral', ls='--', lw=2)
ax1.set_xlabel('Silhouette Score')
ax1.set_ylabel('Frequency')
ax1.set_title('Silhouette Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Silhouette vs min_cluster_size
ax2 = axes[0, 1]
for et, c, m in [('original', 'steelblue', 'o'), ('normalized', 'coral', 's')]:
    subset = df_valid[df_valid['embedding_type'] == et]
    grouped = subset.groupby('min_cluster_size')['silhouette_score'].max()
    ax2.plot(grouped.index, grouped.values, marker=m, color=c, label=et.title(), lw=2)
ax2.set_xlabel('min_cluster_size')
ax2.set_ylabel('Best Silhouette')
ax2.set_title('Silhouette vs min_cluster_size')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Noise vs Silhouette
ax3 = axes[1, 0]
for et, c in [('original', 'steelblue'), ('normalized', 'coral')]:
    subset = df_valid[df_valid['embedding_type'] == et]
    ax3.scatter(subset['noise_ratio'], subset['silhouette_score'], alpha=0.5, label=et.title(), c=c)
ax3.set_xlabel('Noise Ratio (%)')
ax3.set_ylabel('Silhouette Score')
ax3.set_title('Trade-off: Noise vs Silhouette')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Top 10 bar
ax4 = axes[1, 1]
top10 = df_valid.nlargest(10, 'silhouette_score')
colors = ['steelblue' if t=='original' else 'coral' for t in top10['embedding_type']]
ax4.barh(range(10), top10['silhouette_score'], color=colors)
ax4.set_yticks(range(10))
ax4.set_yticklabels([f"mcs={r['min_cluster_size']}, ms={r['min_samples']}" for _,r in top10.iterrows()], fontsize=8)
ax4.set_xlabel('Silhouette Score')
ax4.set_title('Top 10 Configurations')
ax4.invert_yaxis()
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plot_path = os.path.join(OUTPUT_DIR, 'grid_search_visualization.png')
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"📊 Saved: {plot_path}")
plt.show()

In [ ]:
# ============================================================
# CELL 11: Save Results
# ============================================================

# Save CSV
csv_path = os.path.join(OUTPUT_DIR, "experiment1_extended_results.csv")
df_all.to_csv(csv_path, index=False)
print(f"💾 CSV: {csv_path}")

# Save best result
best_data = {
    'winner': winner,
    'best_params': winner_result['params'],
    'labels': winner_result['labels'],
    'n_clusters': winner_result['n_clusters'],
    'noise_ratio': winner_result['noise_ratio'],
    'silhouette_score': winner_result['silhouette_score'],
    'davies_bouldin_index': winner_result['davies_bouldin_index'],
    'embeddings': winner_emb,
    'metadata': metadata,
    'comparison': {
        'original': {'silhouette': best_orig['silhouette_score'], 'params': best_orig['params']},
        'normalized': {'silhouette': best_norm['silhouette_score'], 'params': best_norm['params']}
    }
}

pkl_path = os.path.join(OUTPUT_DIR, "experiment1_best_result.pkl")
with open(pkl_path, 'wb') as f:
    pickle.dump(best_data, f)
print(f"💾 PKL: {pkl_path}")

# Final Summary
print("\n" + "="*70)
print("📋 FINAL SUMMARY")
print("="*70)
print(f"\n🏆 Winner: {winner}")
print(f"\n🔧 Best Parameters:")
print(f"   - min_cluster_size: {winner_result['params']['min_cluster_size']}")
print(f"   - min_samples: {winner_result['params']['min_samples']}")
print(f"   - cluster_selection_method: {winner_result['params']['cluster_selection_method']}")
print(f"\n📊 Results:")
print(f"   - Clusters: {winner_result['n_clusters']}")
print(f"   - Noise: {winner_result['noise_ratio']:.2f}%")
print(f"   - Silhouette: {winner_result['silhouette_score']:.4f}")
print(f"   - DBI: {winner_result['davies_bouldin_index']:.4f}")
print("="*70)
print("\n✅ EXPERIMENT 1 EXTENDED COMPLETE!")